In [ ]:
def aimPath(patName_ord,reader_root,sequence,reader_ROI):
#     reader_root = Level_3_path
#     sequence = 't1ce_N4_Zscored.nii.gz'
#     reader_ROI = 'Segmentation-1stSketch-label.nii.gz'

    # 获取所有第二级目录下的目标文件
    label_file_path_list = []
    for i in reader_root:
        pat_name = i.split('\\')[patName_ord]   # 被试名在哪个层级
        sequence_path = os.path.join(i,sequence)    # 以哪个序列为底
        label_path =os.path.join(i,reader_ROI)   # ROI文件

        if os.path.exists(label_path) and os.path.exists(sequence_path):
            label_file_path_list.append([pat_name,sequence_path,label_path])
        else:
            print('无目标',i)

    print(len(label_file_path_list))

    for i in label_file_path_list:
        print('---------------------------------------------------------------------')
        print(i)
        
    return(label_file_path_list)

In [ ]:
def radiomics_feature_extraction(input_list,save_name):
    # -------------------------------------------------------------
    # 提取特征前定义提取器
    settings = {}
    settings['binWidth'] = 25  # 25
    settings['sigma'] = [3, 5]

    # 标准化
    settings['normalize'] = True
    settings['normalizeScale'] = 100

    # 重采样
    settings['resampledPixelSpacing'] = [1,1,1]
    settings['interpolator'] = sitk.sitkBSpline

    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
    extractor.enableAllImageTypes()
    extractor.enableAllFeatures()
    
    # parameter force2D must be set to True to enable shape2D extraction
    # settings['force2D'] = True
    
    # ## 其他滤波
    # extractor.enableImageTypeByName('LoG')
    # extractor.enableImageTypeByName('Wavelet')
    # # # 文献里的手动加入的影像组学因子
    # extractor.enableFeaturesByName(firstorder=['Energy', 'TotalEnergy', 'Entropy', 'Minimum', '10Percentile', '90Percentile', 'Maximum', 'Mean', 'Median', 'InterquartileRange', 'Range', 'MeanAbsoluteDeviation', 'RobustMeanAbsoluteDeviation', 'RootMeanSquared', 'Skewness', 'Kurtosis', 'Variance', 'Uniformity'])
    # extractor.enableFeaturesByName(shape=['VoxelVolume', 'MeshVolume', 'SurfaceArea', 'SurfaceVolumeRatio', 'Sphericity','Maximum3DDiameter','Maximum2DDiameterSlice','Maximum2DDiameterColumn','Maximum2DDiameterRow', 'MajorAxisLength', 'MinorAxisLength', 'LeastAxisLength', 'Elongation', 'Flatness'])


    error_fileFolder = []
    df = pd.DataFrame()

    for i in tqdm(range(len(input_list))):     # tqdm是 展示进度的tqdm库,外面只是嵌套了一下,不影响程序运行
        print(f"运行【{input_list[i][0]}】")

        try:
            featureVector = extractor.execute(input_list[i][1], input_list[i][2])

            # 添加自定义的内容
            NameDict = {'Subject':input_list[i][0]}
            featureVector.update(NameDict)

            df_add = pd.DataFrame.from_dict(featureVector.values()).T
            df_add.columns = featureVector.keys()
            df = pd.concat([df, df_add])

        except:
            error_fileFolder.append(input_list[i][0])


        # 进度条展示的时间暂停
        sleep(0.02)
    
    print('███████████████████████████████████')
    print('已经完成')
    df.to_csv(os.path.join(root_path,save_name))
    
    
    print('报错文件夹位置：\n')
    print(error_fileFolder)

In [ ]:
import os
import pandas as pd
from time import sleep
import SimpleITK as sitk
from radiomics import featureextractor

from tqdm import tqdm
# from tqdm.notebook import tqdm  # jupyter 特供

# 获取所有待转换文件，并且标记 被试名
格式为：[('TCGA-CS-539','xxx t2文件的完整路径',xxx label文件的完整路径'),(),()]

In [ ]:
# 获取真正的根目录
root_path = r'D:\mysubject\Seg_jsph'

# 获取根目录下所有文件夹
Level_1_name = next(os.walk(root_path))[1]
Level_1_path = [root_path+os.sep+i for i in Level_1_name]

# ████████████████████████████
# 下一级目录
Level_2_name = []   # 第二级目录的文件名
Level_2_path = []   # 第二级目录的文件目录名
for i in Level_1_path:
    tem = next(os.walk(i))[1]
    # print(f'{i}\n各第二级目录下文件/文件夹为：{tem}\n')
    Level_2_name.append(tem)

    for j in tem:
        Level_2_path.append(i+os.sep+j)

# 所有第二级目录为
print(f'第二级目录个数：{len(Level_2_path)}')
Level_2_path

# ████████████████████████████
# 再进一层
Level_3_path = []
for i in Level_2_path:
    tem = next(os.walk(i))[1]
    Level_3_folder = i+os.sep+tem[0]
    Level_3_path.append(Level_3_folder)

# 所有第三级目录为
print(f'第三级目录个数：{len(Level_3_path)}')
Level_3_path

In [ ]:
# 第一个勾画
readerA_ROI_segmentation = 'Segmentation-1stSketch-label.nii.gz'
readerA = aimPath(-3,
                  Level_3_path,
                  't2_N4-WSed.nii.gz',
                  readerA_ROI_segmentation)

In [ ]:
# 第二个勾画
readerB_ROI_segmentation = 'Segmentation-2ndSketch-label.nii.gz'
readerB = aimPath(-3,
                  Level_3_path,
                  't2_N4-WSed.nii.gz',
                  readerB_ROI_segmentation)

# 特征提取

In [ ]:
# # 试水一个
# readerA = readerA[:2]
# # readerA

In [ ]:
radiomics_feature_extraction(readerA,'JSPH1.csv')

In [ ]:
radiomics_feature_extraction(readerB,'JSPH2.csv')